In [21]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import matplotlib.pyplot as plt
import numpy as np

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [6]:
X_train_normalized, X_test_normalized = X_train / 255.0, X_test / 255.0

## Model

In [61]:
HP_NORMALIZE_INPUT = hp.HParam('normalize_input', hp.Discrete([True, False]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_EPOCH = hp.HParam('epoch', hp.IntInterval(1, 5))
HP_L2_REGULARIZATION = hp.HParam('l2_regularization', hp.Discrete([0.0, 0.0001, 0.001, 0.01, 0.1]))

HPARAMS_LIST = [HP_NORMALIZE_INPUT, HP_OPTIMIZER, HP_EPOCH, HP_L2_REGULARIZATION]

METRIC_ACCURACY = 'accuracy'

TF_LOGDIR = "logs/cs231n_softmax_hparam_tuning"

with tf.summary.create_file_writer(TF_LOGDIR).as_default():
    hp.hparams_config(
      hparams=HPARAMS_LIST,
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [62]:
def train_and_evaluate_model(hparams):
    if hparams[HP_NORMALIZE_INPUT]:
        X_train_processed, X_test_processed = X_train_normalized, X_test_normalized
    else:
        X_train_processed, X_test_processed = X_train, X_test
        
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(
            10, 
            activation="softmax", 
            kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_L2_REGULARIZATION])
        ),
    ])
    
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss="sparse_categorical_crossentropy",
        metrics=[METRIC_ACCURACY]
    )
    
    model.fit(X_train_processed, y_train, epochs=hparams[HP_EPOCH])
    
    _, accuracy = model.evaluate(X_test_processed, y_test, verbose=True)
    
    return accuracy

In [68]:
def run(run_dir, hparams):    
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_and_evaluate_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [69]:
# clear directory
#!rm -rf $TF_LOGDIR

In [70]:
session_num = 0

for normalize_input in HP_NORMALIZE_INPUT.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for epoch in range(HP_EPOCH.domain.min_value, HP_EPOCH.domain.max_value):
            for l2_regularization in HP_L2_REGULARIZATION.domain.values:
                hparams = {
                    HP_NORMALIZE_INPUT: normalize_input,
                    HP_OPTIMIZER: optimizer,
                    HP_EPOCH: epoch,
                    HP_L2_REGULARIZATION: l2_regularization,
                }

                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run(TF_LOGDIR + "/" + run_name, hparams)
                session_num += 1

--- Starting trial: run-0
{'normalize_input': False, 'optimizer': 'adam', 'epoch': 1, 'l2_regularization': 0.0}
313/313 [==============================] - 1s 2ms/step - loss: 133.1143 - accuracy: 0.2176
--- Starting trial: run-1
{'normalize_input': False, 'optimizer': 'adam', 'epoch': 1, 'l2_regularization': 0.0001}
313/313 [==============================] - 1s 2ms/step - loss: 112.6769 - accuracy: 0.1916
--- Starting trial: run-2
{'normalize_input': False, 'optimizer': 'adam', 'epoch': 1, 'l2_regularization': 0.001}
313/313 [==============================] - 1s 2ms/step - loss: 85.1139 - accuracy: 0.2737
--- Starting trial: run-3
{'normalize_input': False, 'optimizer': 'adam', 'epoch': 1, 'l2_regularization': 0.01}
313/313 [==============================] - 1s 2ms/step - loss: 80.5430 - accuracy: 0.2380
--- Starting trial: run-4
{'normalize_input': False, 'optimizer': 'adam', 'epoch': 1, 'l2_regularization': 0.1}
313/313 [==============================] - 1s 3ms/step - loss: 78.5397 -

Epoch 1/4
1563/1563 [==============================] - 3s 2ms/step - loss: 98.4164 - accuracy: 0.2319
Epoch 2/4
1563/1563 [==============================] - 3s 2ms/step - loss: 87.3873 - accuracy: 0.2605
Epoch 3/4
1563/1563 [==============================] - 3s 2ms/step - loss: 84.1039 - accuracy: 0.2688
Epoch 4/4
313/313 [==============================] - 0s 2ms/step - loss: 75.4496 - accuracy: 0.2693
--- Starting trial: run-19
{'normalize_input': False, 'optimizer': 'adam', 'epoch': 4, 'l2_regularization': 0.1}
Epoch 1/4
1563/1563 [==============================] - 3s 2ms/step - loss: 104.2462 - accuracy: 0.2362: 0s - loss: 104.8603 - accura
Epoch 2/4
1563/1563 [==============================] - 3s 2ms/step - loss: 88.8896 - accuracy: 0.2620
Epoch 3/4
1563/1563 [==============================] - 3s 2ms/step - loss: 86.6339 - accuracy: 0.2662
Epoch 4/4
313/313 [==============================] - 0s 2ms/step - loss: 67.4624 - accuracy: 0.2988
--- Starting trial: run-20
{'normalize_input

Epoch 1/4
1563/1563 [==============================] - 3s 2ms/step - loss: 84144.5469 - accuracy: 0.2137
Epoch 2/4
1563/1563 [==============================] - 3s 2ms/step - loss: 73049.9141 - accuracy: 0.2430
Epoch 3/4
1563/1563 [==============================] - 3s 2ms/step - loss: 69932.5703 - accuracy: 0.2507
Epoch 4/4
313/313 [==============================] - 0s 1ms/step - loss: 79366.7266 - accuracy: 0.2276
--- Starting trial: run-37
{'normalize_input': False, 'optimizer': 'sgd', 'epoch': 4, 'l2_regularization': 0.001}
Epoch 1/4
1563/1563 [==============================] - 3s 2ms/step - loss: 83844.7969 - accuracy: 0.2143
Epoch 2/4
1563/1563 [==============================] - 3s 2ms/step - loss: 73031.4766 - accuracy: 0.2394
Epoch 3/4
1563/1563 [==============================] - 3s 2ms/step - loss: 70489.9453 - accuracy: 0.2475
Epoch 4/4
313/313 [==============================] - 0s 2ms/step - loss: 95772.9375 - accuracy: 0.2475
--- Starting trial: run-38
{'normalize_input': Fal

Epoch 1/3
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2716 - accuracy: 0.2707
Epoch 2/3
1563/1563 [==============================] - 4s 2ms/step - loss: 2.1394 - accuracy: 0.2814: 0s - loss: 2.1385 - accura
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 2.2160 - accuracy: 0.2782
--- Starting trial: run-55
{'normalize_input': True, 'optimizer': 'adam', 'epoch': 4, 'l2_regularization': 0.0}
Epoch 1/4
1563/1563 [==============================] - 5s 3ms/step - loss: 1.9651 - accuracy: 0.3165
Epoch 2/4
1563/1563 [==============================] - 4s 2ms/step - loss: 1.8803 - accuracy: 0.3521
Epoch 3/4
1563/1563 [==============================] - 5s 3ms/step - loss: 1.8608 - accuracy: 0.3628
Epoch 4/4
313/313 [==============================] - 1s 2ms/step - loss: 2.0180 - accuracy: 0.3270
--- Starting trial: run-56
{'normalize_input': True, 'optimizer': 'adam', 'epoch': 4, 'l2_regularization': 0.0001}
Epoch 1/4
1563/1563 [=====================

313/313 [==============================] - 1s 2ms/step - loss: 1.9563 - accuracy: 0.3114
--- Starting trial: run-72
{'normalize_input': True, 'optimizer': 'sgd', 'epoch': 3, 'l2_regularization': 0.001}
Epoch 1/3
1563/1563 [==============================] - 7s 5ms/step - loss: 1.9705 - accuracy: 0.3041
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 1.8656 - accuracy: 0.3524
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 1.9397 - accuracy: 0.3318
--- Starting trial: run-73
{'normalize_input': True, 'optimizer': 'sgd', 'epoch': 3, 'l2_regularization': 0.01}
Epoch 1/3
1563/1563 [==============================] - 5s 3ms/step - loss: 2.1156 - accuracy: 0.3002
Epoch 2/3
1563/1563 [==============================] - 4s 2ms/step - loss: 1.9683 - accuracy: 0.3467
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 1.9545 - accuracy: 0.3241
--- Starting trial: run-74
{'normalize_input': True, 'optimizer': 'sgd', 'epoch': 3,

## Analysis

In [71]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [72]:
%tensorboard --logdir $TF_LOGDIR

Reusing TensorBoard on port 6006 (pid 34043), started 0:17:28 ago. (Use '!kill 34043' to kill it.)